## 2. Gibbs Sampler

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse
import matplotlib.transforms as transforms

def plot_gaussian_from_points(x, y, ax, n_std=3.0, facecolor='none', **kwargs):
    """
    Create a plot of the covariance confidence ellipse of *x* and *y*.
    Parameters
    ----------
    x, y : array-like, shape (n, )
        Input data.
    ax : matplotlib.axes.Axes
        The axes object to draw the ellipse into.
    n_std : float
        The number of standard deviations to determine the ellipse's radiuses.
    Returns
    -------
    matplotlib.patches.Ellipse
    Other parameters
    ----------------
    kwargs : `~matplotlib.patches.Patch` properties
    """
    if len(x) != len(y):
        raise ValueError("x and y must be the same size")
    if len(x) < 2:
        raise ValueError("Need more data.")
    cov = np.cov(x, y)
    pearson = cov[0, 1]/np.sqrt(cov[0, 0] * cov[1, 1])
    # Using a special case to obtain the eigenvalues of this
    # two-dimensionl dataset.
    ell_radius_x = np.sqrt(1 + pearson)
    ell_radius_y = np.sqrt(1 - pearson)
    ellipse = Ellipse((0, 0),
        width=ell_radius_x * 2,
        height=ell_radius_y * 2,
        facecolor=facecolor,
        **kwargs)

    # Calculating the stdandard deviation of x from
    # the squareroot of the variance and multiplying
    # with the given number of standard deviations.
    scale_x = np.sqrt(cov[0, 0]) * n_std
    mean_x = np.mean(x)

    # calculating the stdandard deviation of y ...
    scale_y = np.sqrt(cov[1, 1]) * n_std
    mean_y = np.mean(y)

    transf = transforms.Affine2D() \
        .rotate_deg(45) \
        .scale(scale_x, scale_y) \
        .translate(mean_x, mean_y)

    ellipse.set_transform(transf + ax.transData)
    return ax.add_patch(ellipse)

def plot_gaussian_from_parameters(mean, cov, ax, n_std=3.0, facecolor='none', **kwargs):
    """
    Create a plot of the covariance confidence ellipse of *x* and *y*.
    Parameters
    ----------
	mean : array-like, shape (2, )
    	Mean vector
    cov : array-like, shape (2,2)
    	Covariance matrix
    ax : matplotlib.axes.Axes
        The axes object to draw the ellipse into.
    n_std : float
        The number of standard deviations to determine the ellipse's radiuses.
    Returns
    -------
    matplotlib.patches.Ellipse
    Other parameters
    ----------------
    kwargs : `~matplotlib.patches.Patch` properties
    """
    if len(mean) != 2:
        raise ValueError("Mean vector length should be 2.")
    if (cov.shape != (2, 2)):
    	raise ValueError("Covariance should be a 2x2 matrix.")
    #checking if cov is symmetric pos semidefinite
    if(cov[0, 1] != cov[1, 0]):
        raise ValueError("Covariance should be symmetric.")
    if(cov[0, 0] < 0 or cov[0, 0]*cov[1,1] - cov[0,1]**2 < 0):
        raise ValueError("Covariance should be positive semidefinite.")

    pearson = cov[0, 1]/np.sqrt(cov[0, 0] * cov[1, 1])
    # Using a special case to obtain the eigenvalues of this
    # two-dimensionl dataset.
    ell_radius_x = np.sqrt(1 + pearson)
    ell_radius_y = np.sqrt(1 - pearson)
    ellipse = Ellipse((0, 0),
        width=ell_radius_x * 2,
        height=ell_radius_y * 2,
        facecolor=facecolor,
        **kwargs)

    # Calculating the stdandard deviation of x from
    # the squareroot of the variance and multiplying
    # with the given number of standard deviations.
    scale_x = np.sqrt(cov[0, 0]) * n_std
    mean_x = mean[0]

    # calculating the stdandard deviation of y ...
    scale_y = np.sqrt(cov[1, 1]) * n_std
    mean_y = mean[1]

    transf = transforms.Affine2D() \
        .rotate_deg(45) \
        .scale(scale_x, scale_y) \
        .translate(mean_x, mean_y)

    ellipse.set_transform(transf + ax.transData)
    return ax.add_patch(ellipse)


# from random import random

# x = np.array([random()*5 for i in range(500)])
# y = np.array([random()*5 for i in range(500)])
# fig = plt.figure()
# ax = fig.add_axes([0,0,1,1])
# ax.scatter(x, y)
# print(plot_gaussian_from_points(x, y, ax, n_std=1, edgecolor='red'))
# #print(plot_gaussian_from_parameters(np.array([2.5, 2.5]), np.cov(x, y), ax, n_std=1, edgecolor='red'))
# plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
# import gif
# gif 가 잘 안 읽히신다면 아래 코드로 설치해주세요
!pip install -U gif --user
!pip install "gif[matplotlib]" --user 
import gif
from IPython.display import Image
from random import random

In [ ]:
def conditional_sampler(sampling_index, current_x, mean, cov):
    conditioned_index = 1 - sampling_index # 두 r.v. 중에 뭘 sampling할지 결정
    a = cov[sampling_index, sampling_index] # Sigma00
    b = cov[sampling_index, conditioned_index]  # Sigma01
    c = cov[conditioned_index, conditioned_index]  # Sigma11
    
    mu = mean[sampling_index]+(b*current_x[conditioned_index]-mean[conditioned_index])/c
    sigma = np.sqrt(a-(b**2)/c)
    new_x = np.copy(current_x)
    new_x[sampling_index] = np.random.randn()*sigma + mu
    # [x_0, x_1] 꼴의 1x2 np.array를 return
    return new_x

In [ ]:
def gibbs_sampler(initial_point, num_samples, mean, cov, create_gif=True):
    """
    [input 형태]
    initial_point = [x_0, x_1] = [-9.0, -9.0]
    num_samples = 100
    mean = np.array([0, 0])
    cov = np.array([[10, 3], 
                    [3, 5]])
    """
    frames = []  # for GIF
    point = np.array(initial_point)
    samples = np.empty([num_samples+1, 2])  # sampled points
    samples[0] = point
    tmp_points = np.empty([num_samples, 2]) # inbetween points (중간저장소)

    for i in range(num_samples):
        # 요 for loop이 gibbs sampler 핵심
        # point = [x_0, x_1]
        
        # Sample from p(x_0|x_1)
        point = conditional_sampler(0, point, mean, cov)
        tmp_points[i] = point
        if(create_gif):
            frames.append(plot_samples(samples, i+1, tmp_points, i+1, title="Num Samples: " + str(i)))
            
        # Sample from p(x_1|x_0)
        point = conditional_sampler(1, point, mean, cov)
        samples[i+1] = point
        if(create_gif):
            frames.append(plot_samples(samples, i+2, tmp_points, i+1, title="Num Samples: " + str(i+1)))
            
    if(create_gif):
        return samples, tmp_points, frames
    else:
        return samples, tmp_points

In [ ]:
@gif.frame
def plot_samples(samples, num_samples, tmp_points, num_tmp, title="Gibbs Sampling", xlims=(-11, 11), ylims=(-11, 11)):
    fig = plt.figure(figsize=(10, 8))
    ax = fig.gca()
    
    # Plot the true distribution
    plot_gaussian_from_parameters(mean, cov, ax, n_std=2, edgecolor='g', alpha=0.5, label="True Distribution")
    
    # Plot sampled points
    ax.scatter(samples[:num_samples, 0], samples[:num_samples, 1], c='b', s=10, label="Sampled Points")
    ax.scatter(samples[0, 0], samples[0, 1], marker='*', c='g', s=60, label="Initial Point")
    
    # Plot samples from conditional distribution
    ax.scatter(tmp_points[:num_tmp, 0], tmp_points[:num_tmp, 1], c='r', alpha=0.4, s=5, label="Temporary Points")
    
    # Keeping the axes scales same for good GIFS
    ax.set_xlim(xlims)
    ax.set_ylim(ylims)
    
    # Plot lines
    if(num_tmp > 0):
        ax.plot([samples[num_samples-1, 0], tmp_points[num_tmp-1, 0]], 
                [samples[num_samples-1, 1], tmp_points[num_tmp-1, 1]], c='k', alpha=0.25)
        # Plot estimated Gaussian, ignoring the starting point
        if(num_samples > 2):
            plot_gaussian_from_points(samples[1:num_samples, 0], samples[1:num_samples, 1], 
                                      ax, n_std=2, edgecolor='b', alpha=0.5, label="Estimated Distribution")
    
    ax.legend(loc='upper left')
    ax.set_title(title)


In [ ]:
mean = np.array([0, 0])
cov = np.array([[10, 3], 
                [3, 5]])

In [ ]:
# Plot true distribution
fig = plt.figure(figsize=(10, 8))
ax = fig.gca()
plot_gaussian_from_parameters(mean, cov, ax, n_std=2, edgecolor='g', label="True Distribution")
ax.scatter(mean[0], mean[1], c='g')
ax.set_xlim((-11, 11))
ax.set_ylim((-11, 11))
ax.legend(loc='upper left')
plt.show()

In [ ]:
initial_point = [-9.0, -9.0]
num_samples = 500
samples, tmp_points, frames = gibbs_sampler(initial_point, num_samples, mean, cov, create_gif=True)

In [ ]:
# Creating the GIF
# 초당 한 번 update 할 수 있게 느으으으린 움짤임
gif.save(frames, "gibbs500.gif", duration=500)
Image(filename="gibbs500.gif")

## 3. NA imputation

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stats

import missingno as msno
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
diabetes = pd.read_csv('diabetes.csv')
diabetes.head()

In [ ]:
diabetes.isna().sum()

In [ ]:
# Mark invalid zero values as NaN (null)
temp_cols = ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']  
diabetes[temp_cols] = diabetes[temp_cols].replace(0, np.nan)
print(diabetes.isnull().sum())

In [ ]:
# Visualize the number of missing values as a matrix
fig = msno.matrix(diabetes, inline=False, figsize=(13.33,7.5))

# Add labels
plt.xlabel('Variables', size=24, labelpad=16)
plt.ylabel('Total Observations', size=24, labelpad=16)
plt.tick_params(axis='both', labelsize=16, length=8)

plt.show;

In [ ]:
# Visualize the number of missing values as a heatmap
fig = msno.heatmap(diabetes, inline=False, figsize=(12,8))

# Add labels
plt.xlabel('Variables with Missing Values', size=20, labelpad=24)
plt.ylabel('Variables with Missing Values', size=20, labelpad=24)
plt.tick_params(axis='both', labelsize=12, length=6)

plt.show();

In [ ]:
data = diabetes[['Glucose', 'BloodPressure', 'SkinThickness', 'BMI']]

g = sns.pairplot(data, diag_kind="kde")
g.map_lower(sns.kdeplot, levels=4, color=".2")

### Sampling Scheme

In [ ]:
n = data.shape[0]       # data 수
p = data.shape[1]       # column 수

S = 100                 # number of iteration (몇 번 update 할건지)

# 자주 쓰는 함수
inv = np.linalg.inv     # 역행렬 구하기
t = np.transpose        # transpose

# priors
## prior for mu ~ N(  ,   )
# mean vector mu_0
mu0 = data.mean().to_numpy()    # column mean

# covariance matrix Lambda_0 : 공분산 행렬 만들기
sd0 = mu0 / 2                   # 왜 이렇게 initialize하는지 모르겠지만 암튼 prior란 원래 나름의 믿음에 따라 주는거니까..
# 아마 actual dataset의 dispersion을 고려하거나 해서 그런게 아닐까..?
L0 = np.ones((p,p))*0.1         # Lambda0 in prior    
di = np.diag_indices(p)         # diagonal index indicator
L0[di] = 1                      # 일단 분산을 1로 init
L0 = L0 * np.outer(sd0, sd0)    # off-diagonal terms

## prior for Sigma ~ Ing-Wishart(   ,   )
nu0 = p + 2                     # first param
S0 = (nu0-p-1)*L0

# misc
Sigma = S0                      # mu의 full conditional posterior를 초기화하는데 필요함
fill_data = data.copy()         # imputate할 dataset copy 만들기

O = data.isna().to_numpy()*1    # indicator variable (결측치 있으면 1, 결측치 없으면 0)

# Naive Imputation
for col in fill_data.columns:
    # 일단 mean imputation
    # 아마 계산 시 nan 있어서 생기는 error를 방지하기 위해서 나이브하게 뭐라도 채워놓고 simulation을 돌린것같음.
    fill_data[col].fillna(fill_data[col].mean(), inplace=True)

Mean Imputation  

Pros:

Prevent data loss which results in deletion of rows or columns
Works well with a small dataset and easy to implement.


Cons:

Works only with numerical continuous variables.
Can cause data leakage
Does not factor the 'covariance between features.'

In [ ]:
g_naive = sns.pairplot(fill_data, diag_kind="kde")
g_naive.map_lower(sns.kdeplot, levels=4, color=".2")

### Gibbs Sampler Execution

In [ ]:
for s in range(S):
    # update mu
    ybar = fill_data.mean().to_numpy()
    Ln = inv(inv(L0)+inv(Sigma)*n)        # Lambda_n
    mun = np.matmul(Ln, np.matmul(inv(L0),mu0)+np.matmul(n*inv(Sigma),ybar))       # mu_n
    ## sample mu from full conditional probability given Sigma & y ##
    Mu = stats.multivariate_normal.rvs(mun,Ln,1)      

    # update Sigma
    S_mu = np.matmul(t(fill_data-Mu).to_numpy(),(fill_data-Mu).to_numpy())
    Sn = S0+S_mu
    ## sample Sigma from full conditional probability given mu & y ##
    Sigma = inv(stats.wishart.rvs(nu0+n,inv(Sn),1)

    # update missing data
    for i in range(n):

        # row별로 돌아가면서 1x4 array에 대해서 imputation
        # [T, F, F, T] 등의 형태로 indexing
        a = O[i,]==0        # i번째 row에 결측치 없는 위치
        b = O[i,]==1        # i번째 row에 결측치 있는 위치

        if sum(b)!=0:       # 결측치가 하나라도 있으면 imputation 진행! (결측치 하나도 없으면 sum(b)==0 loop 벗어남)
            # iSa, beta_j에 관한 설명은 좀 어려워서 스킵
            # 궁금하면 FCB p118 (7.10), (7.11) equation 설명 참고
            iSa = inv(Sigma[np.outer(a,a)].reshape(sum(a),sum(a)))
            beta_j = np.matmul(Sigma[np.outer(b,a)].reshape(sum(b),sum(a)), iSa)

            # Covariance Matrix for MVN distribution
            Sigma_j = Sigma[np.outer(b,b)].reshape(sum(b),sum(b)) - np.linalg.multi_dot([Sigma[np.outer(b,a)].reshape(sum(b),sum(a)), iSa, Sigma[np.outer(a,b)].reshape(sum(a),sum(b))])
            # Mean Vector for MVN distribution
            mu_j = Mu[b] + np.matmul(beta_j, t(fill_data.iloc[i, a])-Mu[a])

            # MVN 에서 결측치 있는 위치 (b)만 sampling한 값으로 채우기!
            fill_data.iloc[i,b] = stats.multivariate_normal.rvs(mu_j, Sigma_j, 1)

    if s%10==0:
        print(s,"/",S)


In [ ]:
# # debug statement (각각이 어떤 shape, dimension인지 확인하고 싶으면 주석 해제해서 실행해보기)
# ybar = fill_data.mean().to_numpy()
# ybar
# Ln = inv(inv(L0) + inv(Sigma)*n)
# print(Ln.shape)
# mun = np.matmul(Ln, np.matmul(inv(L0), mu0) + np.matmul(n*inv(Sigma), ybar))
# print(mun.shape)
# Mu = stats.multivariate_normal.rvs(mun, Ln, 1)
# print(Mu)

# Sn = S0 + np.matmul(t(fill_data-Mu).to_numpy(), (fill_data-Mu).to_numpy())
# print(Sn)
# Sigma = inv(stats.wishart.rvs(nu0+n, inv(Sn), 1))
# print(Sigma)

# a = O[33,]==0
# print(a)
# b = O[33,]==1
# print(b)
# iSa = inv(Sigma[np.outer(a,a)].reshape(sum(a),sum(a)))
# print(iSa)  
# beta_j = np.matmul(Sigma[np.outer(b,a)].reshape(sum(b),sum(a)), iSa)
# print(beta_j)
# Sigma_j = Sigma[np.outer(b,b)].reshape(sum(b),sum(b)) - np.linalg.multi_dot([Sigma[np.outer(b,a)].reshape(sum(b),sum(a)), iSa, Sigma[np.outer(a,b)].reshape(sum(a),sum(b))])
# print(Sigma_j)
# mu_j = Mu[b] + np.matmul(beta_j, t(fill_data.iloc[3, a])-Mu[a])
# print(mu_j)

In [ ]:
data

In [ ]:
fill_data

In [ ]:

# Visualize the number of missing values as a matrix
fig = msno.matrix(fill_data, inline=False, figsize=(13.33,7.5))

# Add labels
plt.xlabel('Variables', size=24, labelpad=16)
plt.ylabel('Total Observations', size=24, labelpad=16)
plt.tick_params(axis='both', labelsize=16, length=8)

plt.show;


In [ ]:
g_filled = sns.pairplot(fill_data, diag_kind="kde")
g_filled.map_lower(sns.kdeplot, levels=4, color=".2")

원 데이터의 분포 보존 확인!

Appendix
당연히 이런 수고스러움을 거칠필요 없이 석박사들이 만들어놓은 패키지에 더 좋은 Imputator들이 있다!
간단히 소개하자면

pandas : 위에서 naive하게 column mean으로 채우는 건 pandas의 .fillna를 이용해서 채울 수 있다. 이 외에도 데이터에 시간적인 순서가 있다면 ffill 등을 이용해서 직전 값, 직후의 값으로 채우는 방법도 있다.
sklearn : SimpleImputer가 평균값으로 채우는 방법, IterativeImputer가 MICE를 이용해서 채우는 방법이다.
MICE (Multivariate Imputation by Chained Equations) : 본 노트북과 같은 방법을 mice라고 부른다. 사실 대부분 패키지의 mice가 bayesian imputation, gibbs sampler의 아이디어를 차용한 것은 맞으나 대부분의 mice implementaion은 frequentist 관점에서 주어진 값들에 대해 regression을 활용하여 결측치를 예측하는 방법으로 구현되어 있는듯하다!
물론 어떨 때는 걍 drop하는 게 더 좋을 수도 있다!
각각 사용예시는 medium이나 위의 reference에 많으니 궁금한 사람들은 한 번 해보세요!